Basic code for antibody-based simulations
===========

## basic imports ##

In [1]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from mdtraj.reporters import *

from pathlib import Path
import sys
from sys import stdout
import inspect

In [2]:
debug_local = True#False
local = Path("..").resolve()
code = local / "mm"
data  = local / "data"
input = data / "input"
output = data / "output"

if debug_local and code.exists():
    sys.path.insert(0, code.as_posix())
    print("extending pathes with local yspecies")
    print(sys.path)
    %load_ext autoreload
    %autoreload 2

extending pathes with local yspecies
['/data/sources/antibody-mm/mm', '/data/sources/antibody-mm/notebooks', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python39.zip', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/lib-dynload', '', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages', '/home/antonkulaga/micromamba/envs/antibody-mm/lib/python3.9/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


## Setting simulation up ##

### Loading the file ###

In [3]:
pdb_name = "1adq_short.pdb"
pdb_path = (input / pdb_name).as_posix() 
pdb = PDBFile(pdb_path)
modeller = Modeller(pdb.topology, pdb.positions)
modeller.deleteWater() #for implicit solvent


#### Setting up the force field and system ####

In [4]:
forcefield = ForceField('amber14/protein.ff14SB.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(modeller.topology, nonbondedMethod=app.NoCutoff, constraints=HBonds, implicitSolvent=GBn2)
integrator = LangevinMiddleIntegrator(310*kelvin, 1 / picosecond, 0.002*picoseconds)

#### Setting up simulation ####

In [5]:
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()

#### Adding reporters ####

In [6]:
report_interval = 10000
checkpoint_interval = report_interval * 10

simulation.reporters.append(PDBReporter((output / pdb_name).as_posix(), report_interval))
simulation.reporters.append(StateDataReporter(stdout, checkpoint_interval, potentialEnergy=False, kineticEnergy=False, totalEnergy=False, step=True,temperature=True))
simulation.reporters.append(StateDataReporter((output / '1adq_short.pdb_log.tsv').as_posix(), report_interval, potentialEnergy=False, kineticEnergy=False, totalEnergy=False, step=True,temperature=True))
simulation.reporters.append(HDF5Reporter((output / '1adq_short.pdb_log.h5').as_posix(), report_interval, coordinates=True, time=True, cell=True, potentialEnergy=True, kineticEnergy=True, temperature=True))
simulation.reporters.append(CheckpointReporter((output / '1adq_short_results.chk').as_posix(), checkpoint_interval))

## Running the simulation ##

In [ ]:
steps = 50000000 # to get 10 nanoseconds

In [ ]:
simulation.step(steps)

In [ ]:
simulation.saveState((output / '1adq_short_results.xml').as_posix())
simulation.saveCheckpoint((output / '1adq_short_results.chk').as_posix())